In [ ]:
#pip install --upgrade ipykernel

In [ ]:
!pip install en_core_web_sm

ERROR: Could not find a version that satisfies the requirement en_core_web_sm (from versions: none)
ERROR: No matching distribution found for en_core_web_sm
You should consider upgrading via the 'c:\users\philipp\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
import pickle
import gensim
import pyLDAvis
import pyLDAvis.gensim
import spacy
import pandas as pd
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
import warnings
from pprint import pprint
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
%config InlineBackend.figure_formats = ['retina']
from sklearn.metrics import f1_score
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.metrics import fbeta_score
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Philipp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
c:\users\philipp\appdata\local\programs\python\python39\lib\site-packages\matplotlib_inline\config.py:59: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [ ]:
train = pd.read_csv('train2.csv', sep='\t', encoding='ISO-8859-1')
test = pd.read_csv('test2.csv',sep='\t',encoding='ISO-8859-1')

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['get','say'])

In [ ]:
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [ ]:
def bigrams(words, bi_min=5, tri_min=5):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

In [ ]:
def get_corpus(df):
    """
    Get Bigram Model, Corpus, id2word mapping
    """
    
#   df['body'] = strip_newline(df.body)
    words = list(sent_to_words(df.body))
    words = remove_stopwords(words)
    bigram = bigrams(words)
    bigram = [bigram[review] for review in words]
#     lemma = lemmatization(bigram)
    id2word = gensim.corpora.Dictionary(bigram)
    id2word.filter_extremes(no_below=10, no_above=0.35)
    id2word.compactify()
    corpus = [id2word.doc2bow(body) for body in bigram]
    return corpus, id2word, bigram

In [ ]:
train_corpus, train_id2word, bigram_train = get_corpus(train)

In [ ]:
TOPICS = 3

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_train = gensim.models.ldamulticore.LdaMulticore(
                           corpus=train_corpus,
                           num_topics=TOPICS,
                           id2word=train_id2word,
                           chunksize=100,
                           workers=7, # Num. Processing Cores - 1
                           passes=50,
                           eval_every = 1,
                           per_word_topics=True)
    lda_train.save('lda_train3.model')

In [ ]:
train_vecs = []
for i in range(len(train)):
    top_topics = lda_train.get_document_topics(train_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(TOPICS)]
    train_vecs.append(topic_vec)

In [ ]:
X3 = np.array(train_vecs)

In [ ]:
TOPICS = 20

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_train = gensim.models.ldamulticore.LdaMulticore(
                           corpus=train_corpus,
                           num_topics=TOPICS,
                           id2word=train_id2word,
                           chunksize=100,
                           workers=7, # Num. Processing Cores - 1
                           passes=50,
                           eval_every = 1,
                           per_word_topics=True)
    lda_train.save('lda_train20.model')

In [ ]:
train_vecs = []
for i in range(len(train)):
    top_topics = lda_train.get_document_topics(train_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(TOPICS)]
    train_vecs.append(topic_vec)

In [ ]:
X20 = np.array(train_vecs)

In [ ]:
cols = ['ups', 'score', 'body_length', 'gif', 'wonglik', 'user wonglik',
       'offsided', 'text', 'word', 'reply', 'tip', 'github', 'please contact',
       'tie', 'love', 'news', 'never', 'guy', 'first', 'day', 'game', 'got',
       'also', 'yeah', 'work', 'sure', 'year', 'polarity', 'subjectivity']

In [ ]:
train_vecs = []

for i in range(len(train)):
  topic_vec = []
  for c in cols:
    topic_vec.extend([train.iloc[i][c]])   
  train_vecs.append(topic_vec)

In [ ]:
features = np.array(train_vecs)
X_train = np.concatenate((X3,X20,features), axis=1)
y_train = np.array(train.Y)

In [ ]:
def get_bigram(df):
    words = list(sent_to_words(df.body))
    words = remove_stopwords(words)
    bigram = bigrams(words)
    bigram = [bigram[review] for review in words]
#     lemma = lemmatization(bigram)
    return bigram

In [ ]:
bigram_test = get_bigram(test)

In [ ]:
lda_train3 = gensim.models.ldamulticore.LdaMulticore.load('lda_train3.model')
lda_train20 = gensim.models.ldamulticore.LdaMulticore.load('lda_train20.model')

In [ ]:
test_corpus = [train_id2word.doc2bow(body) for body in bigram_test]

In [ ]:
test_vecs3 = []
for i in range(len(test)):
    top_topics = lda_train3.get_document_topics(test_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(3)]
    test_vecs3.append(topic_vec)

X3 = np.array(test_vecs3)

test_vecs20 = []
for i in range(len(test)):
    top_topics = lda_train20.get_document_topics(test_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(20)]
    test_vecs20.append(topic_vec)

X20 = np.array(test_vecs20)

In [ ]:
test_vecs = []

for i in range(len(test)):
  topic_vec = []
  for c in cols:
    topic_vec.extend([test.iloc[i][c]])   
  test_vecs.append(topic_vec)

In [ ]:
features = np.array(test_vecs)
X_test = np.concatenate((X3,X20,features), axis=1)
y_test = np.array(test.Y)

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score


ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

lr = LogisticRegression(
  class_weight= 'balanced',
  solver='newton-cg',
  fit_intercept=True
  ).fit(X_train_sc, y_train)

y_pred_lr = lr.predict(X_test_sc)
print(f1_score(y_test, y_pred_lr,average='weighted'))

0.6096562178863778


In [ ]:
columns = ['a3','b3','c3','d20','e20','f20','g20','h20','i20','j20','k20','l20','m20','n20','o20','p20','q20','r20','s20','t20','u20','v20','w20','ups', 'score', 'body_length', 'gif', 'wonglik', 'user wonglik',
       'offsided', 'text', 'word', 'reply', 'tip', 'github', 'please contact',
       'tie', 'love', 'news', 'never', 'guy', 'first', 'day', 'game', 'got',
       'also', 'yeah', 'work', 'sure', 'year', 'polarity', 'subjectivity']

In [ ]:
test = pd.DataFrame(X_test, columns = columns)
test['Y'] = y_test

In [ ]:
train = pd.DataFrame(X_train, columns = columns)
train['Y'] = y_train

In [ ]:
test

,a3,b3,c3,d20,e20,f20,g20,h20,i20,j20,...,game,got,also,yeah,work,sure,year,polarity,subjectivity,Y
0,0.333333,0.333333,0.333333,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0
1,0.100901,0.366986,0.532114,0.008334,0.008334,0.008334,0.008334,0.008334,0.008334,0.008334,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0
2,0.111156,0.115186,0.773658,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.416667,0.833333,0
3,0.067100,0.563246,0.369654,0.010002,0.010002,0.010002,0.010002,0.010002,0.010002,0.010002,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0
4,0.031438,0.845717,0.122844,0.004548,0.094991,0.004548,0.004548,0.004548,0.004548,0.004548,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.066667,0.333333,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.023467,0.022336,0.954197,0.004546,0.095442,0.186362,0.004546,0.004546,0.128660,0.004546,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.110000,0.760000,2
2996,0.167912,0.170819,0.661269,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,2
2997,0.300393,0.066733,0.632874,0.012500,0.012500,0.012500,0.012500,0.012500,0.012500,0.012500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.136364,0.454545,2
2998,0.057651,0.014569,0.927780,0.002943,0.061798,0.002943,0.179458,0.002943,0.179495,0.002943,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.041667,0.166667,2


In [ ]:
train

,a3,b3,c3,d20,e20,f20,g20,h20,i20,j20,...,game,got,also,yeah,work,sure,year,polarity,subjectivity,Y
0,0.912207,0.042289,0.045503,0.006251,0.006251,0.006251,0.006251,0.006251,0.006251,0.006251,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0
1,0.019896,0.019896,0.960208,0.002942,0.002942,0.002942,0.002942,0.002942,0.002942,0.002942,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0
2,0.933314,0.033344,0.033342,0.005001,0.005001,0.005001,0.005001,0.005001,0.005001,0.005001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.412500,0.725000,0
3,0.954828,0.021896,0.023276,0.003125,0.003125,0.003125,0.003125,0.003125,0.003125,0.003125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.033333,0.388889,0
4,0.030960,0.711679,0.257361,0.003334,0.003334,0.469940,0.003334,0.003334,0.003334,0.089530,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.250000,0.250000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197063,0.066825,0.066827,0.866348,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.800000,1.000000,2
197064,0.033480,0.033412,0.933108,0.012502,0.012502,0.012502,0.262444,0.012502,0.012502,0.012502,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.500000,0.688889,2
197065,0.066838,0.066840,0.866322,0.016668,0.349967,0.016668,0.016668,0.016668,0.016668,0.016668,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.800000,0.400000,2
197066,0.166674,0.166674,0.666651,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,2


In [ ]:
train.to_csv('train3.csv', sep='\t')
test.to_csv('test3.csv', sep='\t')